# Capstone 2 - Predicting Water Pump Condition in Tanzania Data Munging

Kenneth Liao

---

## Background

The UN publishes and reviews a list of least developed countries (LDC) every 3 years. LDCs are “low-income countries confronting severe structural impediments to sustainable development. They are highly vulnerable to economic and environmental shocks and have low levels of human assets.”$^{1}$. Tanzania has been classified as an LDC since the UN published the first list of LDCs in 1971$^{2}$. A common challenge of LDCs is a lack of infrastructure to support the development of the nation, including access to education and healthcare, waste management, and access to potable water.

According to UNICEF, as of 2017, more than 24 million Tanzanians lacked access to basic drinking water$^{3}$. This corresponds to only 56.7% of the country’s population having access to basic drinking water. Outside of developed urban areas, much of the potable water is accessed via water pumps. 

Taarifa is an open-source platform for crowd-sourced reporting and triaging of infrastructure related issues. Together with the Tanzanian Ministry of Water, data has been collected for thousands of water pumps throughout Tanzania. The goal of this project is to be able to predict the condition of these water pumps to improve maintenance, reduce pump downtime, and ensure basic water access for millions of Tanzanians.

**References**

1. https://www.un.org/development/desa/dpad/least-developed-country-category.html
2. https://www.un.org/development/desa/dpad/wp-content/uploads/sites/45/publication/ldc_list.pdf
3. https://washwatch.org/en/countries/tanzania/summary/statistics/


### Problem Description

Predict the operating condition of water pumps in Tanzania given various metadata on each water pump.

### Strategy

The strategy will be to implement an XGBoost model as well as a neural network model for predictions and compare their performance.

### Data

The dataset is provided by Taarifa, together with the Tanzanian Ministry of Water and is hosted by DrivenData.org:

https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/

---

## XGBOOST MODEL

In [1]:
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import iplot, plot, init_notebook_mode
import plotly.express as px
from config import credentials
from sklearn.model_selection import train_test_split
import xgboost as xgb

init_notebook_mode(connected=True)

In [ ]:
train_data = pd.read_pickle('../data/train_full.pkl')
train_labels = pd.read_csv('../data/train-labels.csv')

In [ ]:
train_data = train_data.reset_index().set_index('id')
train_data['year_recorded'] = pd.to_datetime(train_data.date_recorded).dt.year
train_data['month_recorded'] = pd.to_datetime(train_data.date_recorded).dt.month
train_data['day_recorded'] = pd.to_datetime(train_data.date_recorded).dt.day
train_data = train_data.drop('date_recorded', axis=1)
train_data['years_since_install'] = train_data['year_recorded'] - train_data['construction_year']

In [ ]:
train_data.head()

In [ ]:
train_labels = train_labels.set_index('id')
train_labels = train_labels['status_group'].map({'functional': 0, 'functional needs repair': 1, 'non functional': 2})

In [ ]:
train_labels.head()

In [ ]:
X_train, X_cv, y_train, y_cv = train_test_split(train_data, train_labels, test_size=0.25, random_state=42)

In [ ]:
X_train.to_pickle('../data/X_train.pkl')
X_cv.to_pickle('../data/X_cv.pkl')
y_train.to_pickle('../data/y_train.pkl')
y_cv.to_pickle('../data/y_cv.pkl')

In [2]:
X_train = pd.read_pickle('../data/X_train.pkl')
X_cv = pd.read_pickle('../data/X_cv.pkl')
y_train = pd.read_pickle('../data/y_train.pkl')
y_cv = pd.read_pickle('../data/y_cv.pkl')

In [4]:
dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_cv, label=y_cv)

MemoryError: 

In [ ]:
dtrain.save_binary('dtrain.buffer')
dtest.save_binary('dtest.buffer')

In [ ]:
# specify parameters via map
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }

num_round = 2

bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)